In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import preprocess, accuracy, sparse_mx_to_torch_sparse_tensor


/opt/anaconda3/envs/adv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Nettack performs structure on a targeted node such that after training, the target model performs poorly on the target node.

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Cora in "nettack" setting (ensures proper test split)
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

Loading cora dataset...
Selecting 1 largest connected components


In [13]:
# Train surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item() + 1, nhid=16, with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [14]:
# Pick a target test node (must be correctly classified by the GCN)
target_node = idx_test[0].item()

# Preprocess inputs for Nettack
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)

# Instantiate Nettack
attacker = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
attacker = attacker.to(device)

# Attack the selected node with perturbation budget (e.g., 3)
attacker.attack(features, adj, labels, target_node, n_perturbations=3)

# Get modified adjacency and features
modified_adj = attacker.modified_adj
modified_features = attacker.modified_features


##### Starting attack #####
##### Attack only using structure perturbations #####
##### Attacking the node directly #####
##### Performing 3 perturbations #####
##### ...1/3 perturbations ... #####
##### ...2/3 perturbations ... #####
##### ...3/3 perturbations ... #####


In [15]:
# Re-train a GCN on the original graph
clean_model = GCN(nfeat=features.shape[1], nclass=labels.max().item() + 1, nhid=16, device=device)
clean_model = clean_model.to(device)
clean_model.fit(features, adj_tensor, labels, idx_train)
output_clean = clean_model.predict()
print(f"[Clean Graph] Target node prediction: {output_clean[target_node].argmax().item()}, True label: {labels[target_node].item()}")

# Re-train a GCN on the attacked graph (only for testing the target node)
attacked_model = GCN(nfeat=features.shape[1], nclass=labels.max().item() + 1, nhid=16, device=device)
attacked_model = attacked_model.to(device)
attacked_model.fit(modified_features, modified_adj, labels, idx_train)
output_attacked = attacked_model.predict()
print(f"[Attacked Graph] Target node prediction: {output_attacked[target_node].argmax().item()}, True label: {labels[target_node].item()}")


[Clean Graph] Target node prediction: 2, True label: 2
[Attacked Graph] Target node prediction: 5, True label: 2
